In [1]:
import utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
man, woman = utils.pick_random_files(15, random_state=42)

In [3]:
fs = []
duration = []
autocorr_pitch_mean = []
autocorr_pitch_median = []
cepstrum_pitch_mean = []
cepstrum_pitch_median= []
f1 = []
f2 = []
f3 = []
f4 = []
f_max = []
n_formants = []
speaker = []

In [4]:
width = 21
step = 10
threshold = 5

for sfs, signal in man:
    fs.append(sfs)
    duration.append(signal.size/sfs)
    formants = utils.formants(signal, width, step, sfs)
    auto_pitch = utils.get_pitch(signal, width, step, sfs, threshold, methode=utils.autocorrelation, extend=False)
    cepstrum_pitch = utils.get_pitch(signal, width, step, sfs, threshold, methode=utils.cepstrum, extend=False)
    autocorr_pitch_mean.append(auto_pitch.mean())
    autocorr_pitch_median.append(np.median(auto_pitch))
    cepstrum_pitch_mean.append(cepstrum_pitch.mean())
    cepstrum_pitch_median.append(np.median(cepstrum_pitch))
    f1.append(formants[0])
    f2.append(formants[1])
    f3.append(formants[2])
    f4.append(formants[3])
    f_max.append(formants[-1])
    n_formants.append(formants.size)
    speaker.append('bdl')
    
for sfs, signal in woman:
    fs.append(sfs)
    duration.append(signal.size/sfs)
    formants = utils.formants(signal, width, step, sfs)
    auto_pitch = utils.get_pitch(signal, width, step, sfs, threshold, methode=utils.autocorrelation, extend=False)
    cepstrum_pitch = utils.get_pitch(signal, width, step, sfs, threshold, methode=utils.cepstrum, extend=False)
    autocorr_pitch_mean.append(auto_pitch.mean())
    autocorr_pitch_median.append(np.median(auto_pitch))
    cepstrum_pitch_mean.append(cepstrum_pitch.mean())
    cepstrum_pitch_median.append(np.median(cepstrum_pitch))
    f1.append(formants[0])
    f2.append(formants[1])
    f3.append(formants[2])
    f4.append(formants[3])
    f_max.append(formants[-1])
    n_formants.append(formants.size)
    speaker.append('slt')

In [12]:
d = {'fs': fs, 'duration': duration, 'autocorr_pitch_mean': autocorr_pitch_mean, 'autocorr_pitch_median': autocorr_pitch_median,
     'cepstrum_pitch_mean': cepstrum_pitch_mean, 'cepstrum_pitch_median': cepstrum_pitch_median, 'f1': f1, 'f2': f2, 'f3': f3, 'f4': f4,
     'f_max': f_max, 'n_formants': n_formants, 'speaker': speaker}
df = pd.DataFrame(data=d)

In [13]:
df

,fs,duration,autocorr_pitch_mean,autocorr_pitch_median,cepstrum_pitch_mean,cepstrum_pitch_median,f1,f2,f3,f4,f_max,n_formants,speaker
0,16000,3.305063,1580.222953,695.652174,214.641293,131.147541,104.067018,109.282574,118.821714,153.575350,7087.963868,1290,bdl
1,16000,1.315063,1772.818679,1036.437247,265.161116,227.356069,161.231712,175.162661,176.385802,190.072614,7245.482202,495,bdl
2,16000,2.995062,830.571465,592.592593,247.241679,153.113553,186.721273,187.482363,191.683119,191.781542,7182.390209,1184,bdl
3,16000,3.315062,1179.242166,666.666667,214.398199,141.592920,37.904299,84.016580,100.630453,114.793706,7213.718039,1300,bdl
4,16000,2.525000,1089.054476,516.129032,226.119623,140.350877,56.110641,65.072809,81.906869,118.165794,7169.457145,985,bdl
5,16000,2.475062,1032.728892,640.000000,230.725831,144.144144,87.768114,148.625517,157.442768,164.718245,7264.047113,969,bdl
6,16000,3.365000,1407.210272,640.000000,184.385396,150.943396,63.966273,135.364799,139.717768,153.046852,7108.523270,1315,bdl
7,16000,3.185000,993.367928,627.692308,220.094912,133.333333,17.072939,64.448353,126.115136,138.363135,7415.288215,1255,bdl
8,16000,1.695062,1418.348463,592.592593,259.263918,160.000000,115.514797,126.811176,135.483733,149.637133,7085.468326,658,bdl
9,16000,1.755000,917.909795,500.488759,200.762289,153.846154,104.658507,113.397649,148.495526,162.736767,7044.325111,678,bdl
